# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.agents import tool

In [3]:
@tool
def search(query:str) -> str:
    "search for wether online"
    return "bangalore weather is 37'c"

In [4]:
search.name

'search'

In [5]:
search.description

'search for wether online'

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic import BaseModel,Field

In [9]:
class SearchInput(BaseModel):
    query : str = Field(description=" thing to search for")

In [10]:
@tool(args_schema=SearchInput)
def search(query :str)->str:
    "search for weather online"
    return "bangalore weather is 27'c"


In [11]:
search.args

{'query': {'description': ' thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [12]:
search.run("sf")

"bangalore weather is 27'c"

In [13]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [14]:
from langchain.tools.render import format_tool_to_openai_function

In [15]:
format_tool_to_openai_function(get_current_temperature)

C:\Users\Lalith Rao\AppData\Local\Temp\ipykernel_32364\540789164.py:1: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_current_temperature)


{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [16]:
get_current_temperature({"latitude": 13, "longitude": 14})

C:\Users\Lalith Rao\AppData\Local\Temp\ipykernel_32364\2143878976.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_current_temperature({"latitude": 13, "longitude": 14})
C:\Users\Lalith Rao\AppData\Local\Temp\ipykernel_32364\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 31.8°C'

In [18]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [19]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [21]:
search_wikipedia("Manchester city fc")

"Page: Manchester City F.C.\nSummary: Manchester City Football Club is a professional football club based in Manchester, England, that competes in the Premier League, the top flight of English football. Founded in 1880 as St. Mark's (West Gorton), they became Ardwick Association Football Club in 1887 and Manchester City in 1894. The club's home ground is the Etihad Stadium in east Manchester, to which they moved in 2003, having played at Maine Road since 1923. Manchester City adopted their sky blue home shirts in 1894, the first season with the current name. Over the course of its history, the club has won ten league titles, seven FA Cups, eight League Cups, seven FA Community Shields, one UEFA Champions League, one European Cup Winners' Cup, one UEFA Super Cup and one FIFA Club World Cup.\nThe club joined the Football League in 1892, and won their first major honour, the FA Cup, in 1904. Manchester City had its first major period of success in the late 1960s and early 1970s, winning t

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [38]:
from langchain.chat_models import ChatOpenAI

In [39]:
functions = [
    format_tool_to_openai_function(f) for f in [search_wikipedia,get_current_temperature]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

C:\Users\Lalith Rao\AppData\Local\Temp\ipykernel_32364\1705998481.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0).bind(functions=functions)


In [41]:
model.invoke("what is the weather in BLR right now")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":12.9716,"longitude":77.5946}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1dd0127a800> JSON: {
  "prompt_tokens": 106,
  "completion_tokens": 26,
  "total_tokens": 132,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7bfa27a2-3a39-48f7-aa35-390534f7c1bd-0')

In [42]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query":"Langchain"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1dd0127af30> JSON: {
  "prompt_tokens": 101,
  "completion_tokens": 17,
  "total_tokens": 118,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2255ecea-9288-4bf6-b7f6-7d3751c5c130-0')

In [43]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [44]:
chain.invoke({"input": "what is the weather in BLR right now"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":12.9716,"longitude":77.5946}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1dd0127b700> JSON: {
  "prompt_tokens": 114,
  "completion_tokens": 26,
  "total_tokens": 140,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9de29dad-0b4a-4de8-8db9-aa7f0688055d-0')

In [45]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [46]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [47]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [48]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [49]:
result.tool

'get_current_temperature'

In [50]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [51]:
get_current_temperature(result.tool_input)

C:\Users\Lalith Rao\AppData\Local\Temp\ipykernel_32364\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 12.7°C'

In [54]:
chain.invoke({"input":"hi"})

AgentFinish(return_values={'output': 'Well, hello there! How can I assist you today?'}, log='Well, hello there! How can I assist you today?')

In [56]:
chain.invoke({"input": "what is the weather in sf right now"}) # this is an agent action not response

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":37.7749,"longitude":-122.4194}'}}, response_metadata={'token_usage': <OpenAIObject at 0x1dd015afca0> JSON: {
  "prompt_tokens": 113,
  "completion_tokens": 26,
  "total_tokens": 139,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-a436030e-8afe-4eb5-a8fe-5ae75357e869-0')])

In [57]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [58]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [59]:
result = chain.invoke({"input": "What is the weather in dubai right now?"})

C:\Users\Lalith Rao\AppData\Local\Temp\ipykernel_32364\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [60]:
result

'The current temperature is 23.8°C'

In [61]:
result = chain.invoke({"input": "What is langchain?"})

In [62]:
result

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Milvus (vector database)\nSummary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.\nMilvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.\n\n\n\nPage: Intelligent agent\nSummary: In artificial intelligence, an intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and may improve its performance through machine learning or by acquiring knowledge. Leading AI textbooks define artificial intelligence as the "study and design of intelligent agents," emphasiz

In [63]:
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'